<a href="https://colab.research.google.com/github/HWANG-HOE-SUN/Univ.Project/blob/main/%EC%83%81%EA%B6%8C%EC%83%81%EC%83%9D/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%B3%84%EB%8F%99%EB%A7%A4%EC%B9%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install CRS

     |████████████████████████████████| 6.3 MB 5.9 MB/s 


In [3]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 15.4 MB 41.6 MB/s 


In [4]:
!pip install tqdm

In [5]:
!pip install earthpy

     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 19.3 MB 6.5 MB/s 


In [7]:
import io
import os
import pandas as pd
import numpy as np
# 위-경도 좌표변환을 위함
from pyproj import CRS              
from pyproj import Transformer
from pyproj import Proj, transform
# 반복문 진행률 시각화
import time
from tqdm import tqdm 
# 데이터 시각화
import seaborn as sns
import matplotlib.pyplot as plt
# k-means 군집화 알고리즘
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples

In [8]:
df = pd.read_excel('/content/클러스터별 동 매칭.xlsx')
df.head()

,행 레이블,평균 : 위도,최대값 : 위도,최소값 : 위도2,평균 : 경도2,최대값 : 경도,최소값 : 경도,Unnamed: 7,행 레이블.1,평균 : 경도,최대값 : 경도2,최소값 : 경도.1,평균 : 위도.1,최대값 : 위도.1,최소값 : 위도2.1
0,0,37.554894,37.582350,37.536319,126.927460,126.960901,126.886679,NaN,공덕동,126.957057,126.963677,126.951604,37.548438,37.558341,37.541986
1,1,37.552822,37.577397,37.542246,126.926245,126.957520,126.890828,NaN,대흥동,126.940794,126.946711,126.933960,37.552303,37.556533,37.545165
2,2,37.552189,37.568276,37.536296,126.928277,126.953611,126.897257,NaN,도화동,126.948548,126.955724,126.943916,37.539468,37.542836,37.535121
3,3,37.555808,37.585033,37.538644,126.919680,126.959182,126.879857,NaN,망원1동,126.905740,126.911247,126.893933,37.555158,37.557736,37.552023
4,4,37.553171,37.580154,37.536319,126.924702,126.957861,126.883191,NaN,망원2동,126.903383,126.907899,126.896826,37.559094,37.562559,37.555942


In [9]:
clusters = df.iloc[:302,1:7]
dongs = df.iloc[:16,8:]
clusters.head(2), dongs.head(2)

(     평균 : 위도   최대값 : 위도  최소값 : 위도2    평균 : 경도2    최대값 : 경도    최소값 : 경도
 0  37.554894  37.582350  37.536319  126.927460  126.960901  126.886679
 1  37.552822  37.577397  37.542246  126.926245  126.957520  126.890828,
   행 레이블.1     평균 : 경도   최대값 : 경도2  ...  평균 : 위도.1  최대값 : 위도.1  최소값 : 위도2.1
 0     공덕동  126.957057  126.963677  ...  37.548438   37.558341    37.541986
 1     대흥동  126.940794  126.946711  ...  37.552303   37.556533    37.545165
 
 [2 rows x 7 columns])

In [25]:
!pip install haversine

In [26]:
from haversine import haversine

In [ ]:
donglist = []

for idx,i in tqdm(enumerate(clusters.iloc[:,[0,3]].values)):
  dong_xy = i
  haverlist = []
  for dong in dongs.iloc[:,[1,4]].values: # 동 하나하나의 값과 비교
    haverlist.append(haversine(dong_xy, dong , unit='km'))
    print(np.argmin(haverlist))
    dong_name = dongs.iloc[:,[0]].loc[np.argmin(haverlist)].values  # 군집에서 최소거리인(가장 가까운) 동 이름
    donglist.append(dong_name)

In [171]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map(location=[37.5,126.9])

list1 = []
list2 = []
marker_cluster = MarkerCluster().add_to(map)

# 신한카드+블록경계도, 블록 위-경도 기준 소비지도 그려보기
for a in dongs.iloc[:,[1,4]].index:
    folium.Marker(location = [dongs.iloc[:,[1,4]].loc[a,'평균 : 위도.1'],dongs.iloc[:,[1,4]].loc[a,'평균 : 경도']]).add_to(marker_cluster)
    list1.append(dongs.iloc[:,[1,4]].loc[a,'평균 : 위도.1']) # 위도
    list2.append(dongs.iloc[:,[1,4]].loc[a,'평균 : 경도']) # 경도
map

In [161]:
import folium
from folium.plugins import MarkerCluster

map2 = folium.Map(location=[37.5,126.9])

list1 = []
list2 = []
marker_cluster = MarkerCluster().add_to(map2)

# 신한카드+블록경계도, 블록 위-경도 기준 소비지도 그려보기
for a in clusters.iloc[:,[0,3]].index:
    folium.Marker(location = [clusters.iloc[:,[0,3]].loc[a,'평균 : 위도'],clusters.iloc[:,[0,3]].loc[a,'평균 : 경도2']]).add_to(marker_cluster)
    list1.append(clusters.iloc[:,[0,3]].loc[a,'평균 : 위도']) # 위도
    list2.append(clusters.iloc[:,[0,3]].loc[a,'평균 : 경도2']) # 경도
map2 